# Import Required Libraries
This cell imports necessary libraries for model creation, data preprocessing, and evaluation.

In [2]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import json
import numpy as np
from tensorflow.keras.models import load_model
import os
from glob import glob
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(268, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    "./train",
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=16
)

train_classes = train_generator.class_indices.keys()

val_generator = val_datagen.flow_from_directory(
    "./valid",
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=16,
    classes=list(train_classes)
)

num_classes = len(train_generator.class_indices)

Found 3328 images belonging to 268 classes.
Found 643 images belonging to 268 classes.


In [ ]:
model.fit(
      train_generator,
      validation_data=val_generator,
      epochs=50
  )

Epoch 1/50


c:\Users\ali_e\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


208/208 ━━━━━━━━━━━━━━━━━━━━ 232s 1s/step - accuracy: 0.0453 - loss: 5.4188 - val_accuracy: 0.2862 - val_loss: 4.3747
Epoch 2/50
208/208 ━━━━━━━━━━━━━━━━━━━━ 215s 1s/step - accuracy: 0.2839 - loss: 4.1595 - val_accuracy: 0.5801 - val_loss: 3.1879
Epoch 3/50
208/208 ━━━━━━━━━━━━━━━━━━━━ 0s 888ms/step - accuracy: 0.5093 - loss: 3.0625

In [ ]:
model.save('muzzle.keras')

In [ ]:
model = load_model("muzzle.keras")
embedding_model = Model(inputs=model.input, outputs=model.layers[-2].output)

In [ ]:
def get_embedding(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    embedding = embedding_model.predict(img_array)
    return embedding.flatten()

In [ ]:
def build_embedding_database(train_dir="./train"):
    database = {"embeddings": [], "labels": []}
    for class_name in os.listdir(train_dir):
        class_path = os.path.join(train_dir, class_name)
        if not os.path.isdir(class_path): continue
        for img_path in glob(os.path.join(class_path, "*.jpg")):
            emb = get_embedding(img_path)
            database["embeddings"].append(emb)
            database["labels"].append(class_name)
    return database

In [ ]:
def predict_identity(img_path, database, seuil=0.7):
    query_emb = get_embedding(img_path)
    sims = cosine_similarity([query_emb], database["embeddings"])[0]
    best_score = np.max(sims)
    best_index = np.argmax(sims)
    
    if best_score < seuil:
        return "INCONNUE", best_score
    else:
        return database["labels"][best_index], best_score

In [ ]:
# database = build_embedding_database("./train")

In [ ]:
def add_new_cow_to_database(cow_id, image_paths, database):
    for img_path in os.listdir(image_paths):
        emb = get_embedding(os.path.join(image_paths, img_path))
        database["embeddings"].append(emb)
        database["labels"].append(cow_id)